In [145]:
from tensorflow import keras
import database_connect as db_connect
import pandas as pd
from datetime import timedelta
import numpy as np

In [146]:
CONFIG = db_connect.load_config()

In [147]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [148]:
columns = ['id',
    'hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'mon_sin',
 'mon_cos',
 'weekday_sin',
 'weekday_cos',
 'year',
 'RATIO_close_and_MA3',
 'RATIO_close_and_MA6',
 'RATIO_close_and_MA12',
 'RATIO_close_and_MA24',
 'RATIO_close_and_MA48',
 'RATIO_close_and_MA96',
 'RATIO_close_and_MA192',
 'RATIO_close_and_MA384',
 'RATIO_quote_asset_volume_and_MA3',
 'RATIO_quote_asset_volume_and_MA6',
 'RATIO_quote_asset_volume_and_MA12',
 'RATIO_quote_asset_volume_and_MA24',
 'RATIO_quote_asset_volume_and_MA48',
 'RATIO_quote_asset_volume_and_MA96',
 'RATIO_quote_asset_volume_and_MA192',
 'RATIO_quote_asset_volume_and_MA384',
 'RATIO_num_trades_and_MA3',
 'RATIO_num_trades_and_MA6',
 'RATIO_num_trades_and_MA12',
 'RATIO_num_trades_and_MA24',
 'RATIO_num_trades_and_MA48',
 'RATIO_num_trades_and_MA96',
 'RATIO_num_trades_and_MA192',
 'RATIO_num_trades_and_MA384',
 'RATIO_close_and_STD3',
 'RATIO_close_and_STD6',
 'RATIO_close_and_STD12',
 'RATIO_close_and_STD24',
 'RATIO_close_and_STD48',
 'RATIO_close_and_STD96',
 'RATIO_close_and_STD192',
 'RATIO_close_and_STD384',
 'RSI_3_close',
 'RSI_6_close',
 'RSI_12_close',
 'RSI_24_close',
 'RSI_48_close',
 'RSI_96_close',
 'RSI_192_close',
 'RSI_384_close', 
 'predicted']

In [172]:
model = keras.models.load_model('models/08-02-2024 01-54-51 val_PR 0.995 time frame 4.HD5', compile=False)

In [173]:
TIME_STEP = 4

In [174]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 4, 49)]           0         
                                                                 
 normalization_2 (Normaliza  (None, 4, 49)             99        
 tion)                                                           
                                                                 
 lstm_4 (LSTM)               (None, 4, 49)             19404     
                                                                 
 lstm_5 (LSTM)               (None, 6)                 1344      
                                                                 
 dense_4 (Dense)             (None, 6)                 42        
                                                                 
 dense_5 (Dense)             (None, 4)                 28        
                                                           

In [175]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [176]:
sql = 'select min(timestamp) from trading_bot_historydataeth \
        where predicted is null and hour_sin is not null \
        limit 1'

In [177]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchone()

In [178]:
result = result[0]

In [179]:
result_str = result.strftime('%Y-%m-%d %H:%M:%S')
result_str

'2024-01-09 07:00:00'

In [180]:
sql = 'SELECT "'
sql += '", "'.join(columns)
sql += '"'

In [181]:
sql

'SELECT "id", "hour_sin", "hour_cos", "day_sin", "day_cos", "mon_sin", "mon_cos", "weekday_sin", "weekday_cos", "year", "RATIO_close_and_MA3", "RATIO_close_and_MA6", "RATIO_close_and_MA12", "RATIO_close_and_MA24", "RATIO_close_and_MA48", "RATIO_close_and_MA96", "RATIO_close_and_MA192", "RATIO_close_and_MA384", "RATIO_quote_asset_volume_and_MA3", "RATIO_quote_asset_volume_and_MA6", "RATIO_quote_asset_volume_and_MA12", "RATIO_quote_asset_volume_and_MA24", "RATIO_quote_asset_volume_and_MA48", "RATIO_quote_asset_volume_and_MA96", "RATIO_quote_asset_volume_and_MA192", "RATIO_quote_asset_volume_and_MA384", "RATIO_num_trades_and_MA3", "RATIO_num_trades_and_MA6", "RATIO_num_trades_and_MA12", "RATIO_num_trades_and_MA24", "RATIO_num_trades_and_MA48", "RATIO_num_trades_and_MA96", "RATIO_num_trades_and_MA192", "RATIO_num_trades_and_MA384", "RATIO_close_and_STD3", "RATIO_close_and_STD6", "RATIO_close_and_STD12", "RATIO_close_and_STD24", "RATIO_close_and_STD48", "RATIO_close_and_STD96", "RATIO_close

In [182]:
sql = sql + f" FROM trading_bot_historydataeth \
        WHERE timestamp >= '{result_str}' \
        ORDER BY timestamp;"

In [183]:
with conn.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()

In [184]:
df_predict = pd.DataFrame(result, columns=columns)

In [185]:
df_predict.head()

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close,predicted
0,38191,0.965926,-0.258819,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,42.355637,25.842697,45.334686,20.813048,65.919438,56.021129,53.489367,50.631948,50.175543,None
1,38879,0.866025,-0.500000,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,42.074510,17.234919,25.308349,21.558245,62.743298,55.333333,52.482071,50.122133,49.978709,None
2,38880,0.707107,-0.707107,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,42.077034,23.438281,27.393075,20.970405,65.351531,56.063497,51.729572,49.757115,49.932745,None
3,38881,0.500000,-0.866025,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,42.095640,6.812652,16.705607,21.449312,63.335895,55.801198,53.468366,49.722059,50.057236,None
4,38882,0.258819,-0.965926,0.951057,-0.309017,0.5,0.866025,0.781831,0.62349,2024,...,42.210414,100.000000,35.352649,29.790763,62.144438,56.428631,53.757792,49.835741,50.148873,None


In [186]:
tensor = dataframe_to_tensor(df_predict.drop (columns= ['id', 'predicted']), TIME_STEP)

In [187]:
tensor.shape

(697, 4, 49)

In [189]:
for i in range(tensor.shape[0]):
    res = model.predict(tensor[i, :, :].reshape( 1, TIME_STEP, 49), verbose=0)
    print(res)

[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.25224257]]
[[0.37525803 0.21221739 0.16028208 0.252

In [188]:
model.predict(tensor)

22/22 [==============================] - 0s 3ms/step


array([[0.37525803, 0.21221739, 0.16028208, 0.25224257],
       [0.37525803, 0.21221739, 0.16028208, 0.25224257],
       [0.37525803, 0.21221739, 0.16028208, 0.25224257],
       ...,
       [0.37525803, 0.21221739, 0.16028208, 0.25224257],
       [0.37525803, 0.21221739, 0.16028208, 0.25224257],
       [0.37525803, 0.21221739, 0.16028208, 0.25224257]], dtype=float32)